This network is to learn and then diagnos skin lessions'

In [176]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import ImageFile
import time
import csv
import pandas

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# set this to your local data path for training
img_path_data = 'C:\\Users\\Philip\\Documents\\Anaconda-Python-Scripts\\Course-Udacity-DeepLearning\\deep-learning-v2-pytorch-master\\09-dermatologist-ai\\data\\'


In [177]:
#Load the data.

img_resize = 224  # Minimum image size for the trained Torchvision models is 224
trim_amnt  = 0 # Amount of border to be trimmed from the images
batch_size = 10   # You may need to adjust this if you run out of RAM

#normalization mean & std as required for the torchvision models

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(img_resize),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(int(img_resize*(1+trim_amnt))),
                                      transforms.CenterCrop(img_resize),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
'''

# Assume these images are carefully framed
train_transforms = transforms.Compose([transforms.Resize(img_resize),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(img_resize),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
'''

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(img_path_data+'train', transform=train_transforms)
valid_data = datasets.ImageFolder(img_path_data+'valid', transform=test_transforms)
test_data  = datasets.ImageFolder(img_path_data+'test' , transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)
test_loader  = torch.utils.data.DataLoader(test_data,  batch_size=batch_size)


# Save the loaders in a set
loaders = {'train': train_loader, 
           'valid': valid_loader,
           'test' : test_loader }

#print (len(loaders['train'].dataset.classes))
#print (loaders['train'].batch_size)
#print (loaders['train'].dataset.classes)
#print (loaders['train'].dataset.imgs[0])

In [187]:
#Define the CNN Achitecture

# Load the pretrained model from pytorch
model = models.vgg16(pretrained=True)
#model = models.resnet18(pretrained=True)
#model = models.densenet121(pretrained=True)

# Freeze training for all "features" layers
for param in model.parameters():
    param.requires_grad_(False)

# Get the original number of inputs into the final layer
# For vgg16: model.classifier[6]
# For resnet18: model.fc
# For densnet121: classifier
n_inputs = model.classifier[6].in_features

# Create a inear layer with the correct number of outputs. New layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, len(loaders['train'].dataset.classes))

# Update and replace the final layer
model.classifier[6] = last_layer
    
if use_cuda:
    model = model.cuda()
   
# check to see that your last layer produces the expected number of outputs
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [191]:
# These things  :)
#criterion = nn.NLLLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.003)

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr=0.001, momentum = 0.9)


In [192]:
# Train the network

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    t0 = time.time()
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
           
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tElapsed minutes: {:.1f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            (time.time()-t0)/60
            ))
        
        ## Save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model


# train the model
model = train(50, loaders, model, optimizer, criterion, use_cuda, 'model.pt')

# load the model that got the best validation accuracy
model.load_state_dict(torch.load('model.pt'))

KeyboardInterrupt: 

In [ ]:
# test the netowork

def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    # Initialize the lists for the csv file
    Id = ['Id']
    task_1 = ['task_1']
    task_2 = ['task_2']
    
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
        # Build CSV lists
        output = output.data.numpy()
        for i in range(batch_size):
            id_str = loaders['test'].dataset.imgs[batch_idx*batch_size+i][0]
            id_str = id_str.replace(img_path_data, 'data\\')
            id_str = id_str.replace('\\','/')
            Id.append (id_str)
            task_1.append(output[i][0]) # melanoma
            task_2.append(output[i][2]) # seborrheic_keratosis
            
            
    # Create and save the save the CSV file       
    df = pandas.DataFrame(data={"Id": Id, "task_1": task_1, "task_2": task_2})
    df.to_csv("./results.csv", sep=',',index=False) 
    
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders, model, criterion, use_cuda)